## Bribes research

In [42]:
import math

import pandas as pd
import requests
from IPython.core.display_functions import display

from notebooks.bribes_research.helpers import fetch_token_prices

BADGER_BRIBE = 40000

BRIBES_URL = "https://api.llama.airforce/bribes"
DASHBOARD_URL = "https://api.llama.airforce//dashboard"
TABLE_COLUMNS = ["Protocol", "BADGER total bribe in $", "Bribes Portion %", "Bribes Portion", "Dilution", "$I/O",]
PROTOCOL_CVX = "CVX"
PROTOCOL_AURA = "AURA"

table_rows = []
token_prices = fetch_token_prices(['badger-dao'])
badger_price = token_prices['badger-dao']['usd']
badger_bribe_in_dollar = BADGER_BRIBE * badger_price

cvx_bribes = requests.post(BRIBES_URL, json={'platform': "votium", 'protocol': "cvx-crv"}).json()
cvx_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-votium"}).json()
aura_bribes = requests.post(BRIBES_URL, json={'platform': "hh", 'protocol': "aura-bal"}).json()
aura_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-aura"}).json()

total_bribes_cvx = sum([bribe['amountDollars'] for bribe in cvx_bribes['epoch']['bribes']])
dollar_spent_on_bribes_cvx = cvx_dash['dashboard']['rewardPerDollarBribe']

total_bribes_aura = sum([bribe['amountDollars'] for bribe in aura_bribes['epoch']['bribes']])
dollar_spent_on_bribes_aura = aura_dash['dashboard']['rewardPerDollarBribe']

cvx_dilution_100 = badger_bribe_in_dollar / total_bribes_cvx * 100
aura_dilution_100 = badger_bribe_in_dollar / total_bribes_aura * 100

cvx_in_out_100 = (100 - cvx_dilution_100) * dollar_spent_on_bribes_cvx
table_rows.append(
    [PROTOCOL_CVX, badger_bribe_in_dollar, (badger_bribe_in_dollar / badger_bribe_in_dollar) * 100, badger_bribe_in_dollar, cvx_dilution_100, cvx_in_out_100]
)
aura_in_out_100 = (100 - aura_dilution_100) * dollar_spent_on_bribes_aura
table_rows.append(
    [PROTOCOL_AURA, badger_bribe_in_dollar, (badger_bribe_in_dollar / badger_bribe_in_dollar) * 100, badger_bribe_in_dollar, aura_dilution_100, aura_in_out_100]
)

# Rebalancing bribe values
cvx_dilution_50 = ((badger_bribe_in_dollar / 2) / total_bribes_cvx * 100)
aura_dilution_50 = ((badger_bribe_in_dollar / 2) / total_bribes_aura * 100)

cvx_in_out_50 = (100 - cvx_dilution_50) * dollar_spent_on_bribes_cvx
table_rows.append(
    [PROTOCOL_CVX, badger_bribe_in_dollar, ((badger_bribe_in_dollar / 2) / badger_bribe_in_dollar) * 100, badger_bribe_in_dollar / 2, cvx_dilution_50, cvx_in_out_50]
)
aura_in_out_50 = (100 - aura_dilution_50) * dollar_spent_on_bribes_aura
table_rows.append(
    [PROTOCOL_AURA, badger_bribe_in_dollar, ((badger_bribe_in_dollar / 2) / badger_bribe_in_dollar) * 100, badger_bribe_in_dollar / 2, aura_dilution_50, aura_in_out_50]
)

# Rebalance here
cvx_in_out_floating = cvx_in_out_50
aura_in_out_floating = aura_in_out_50
rebalancing_step = 0.01  # in %
cvx_bribe_portion = aura_bribe_portion = badger_bribe_in_dollar / 2
while not math.isclose(cvx_in_out_floating, aura_in_out_floating, abs_tol=0.5):
    if cvx_in_out_floating > aura_in_out_floating:
        # Take bribe portion from aura and give it to cvx
        bribe_stolen = aura_bribe_portion * rebalancing_step
        aura_bribe_portion -= bribe_stolen
        cvx_bribe_portion += bribe_stolen
    elif aura_in_out_floating > cvx_in_out_floating:
        # Take bribe portion from cvx and give it to aura
        bribe_stolen = aura_bribe_portion * rebalancing_step
        cvx_bribe_portion -= bribe_stolen
        aura_bribe_portion += bribe_stolen
    cvx_dilution =  cvx_bribe_portion / total_bribes_cvx * 100
    aura_dilution = aura_bribe_portion / total_bribes_aura * 100
    cvx_in_out_floating = (100 - cvx_dilution) * dollar_spent_on_bribes_cvx
    aura_in_out_floating = (100 - aura_dilution) * dollar_spent_on_bribes_aura

table_rows.append(
    [PROTOCOL_CVX, badger_bribe_in_dollar, round(cvx_bribe_portion / badger_bribe_in_dollar * 100, 2), cvx_bribe_portion, cvx_dilution, cvx_in_out_floating]
)
table_rows.append(
    [PROTOCOL_AURA, badger_bribe_in_dollar, round(aura_bribe_portion / badger_bribe_in_dollar * 100, 2), aura_bribe_portion, aura_dilution, aura_in_out_floating]
)
dataframe = pd.DataFrame(data=table_rows, columns=TABLE_COLUMNS)
display(dataframe)

CVX $I/O if consuming 100% bribes is 137.77948467480584%
AURA $I/O if consuming 100% bribes is 110.90777188396157%
CVX $I/O if consuming 50% bribes is 142.47567155287564%
AURA $I/O if consuming 50% bribes is 179.7282017155373%
Estimated BADGER bribe for CVX: 42478.797048896806$. Percent from total bribes: 24.81%. $I/O: 144.84138682700592
Estimated BADGER bribe for AURA: 128721.2029511032$. Percent from total bribes: 75.19%. $I/O: 145.05974701986517


,Protocol,BADGER total bribe in $,Bribes Portion %,Bribes Portion,Dilution,$I/O
0,CVX,171200.0,100.00,171200.000000,6.381909,137.779485
1,AURA,171200.0,100.00,171200.000000,55.377838,110.907772
2,CVX,171200.0,50.00,85600.000000,3.190954,142.475672
3,AURA,171200.0,50.00,85600.000000,27.688919,179.728202
4,CVX,171200.0,24.81,42478.797049,1.583504,144.841387
5,AURA,171200.0,75.19,128721.202951,41.637278,145.059747
